# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm
#from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

In [2]:
#bb.set_host_only(True)
print(bb.get_device_name(0))
bb.set_device(0)

NVIDIA GeForce GTX 1660 SUPER


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [3]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLutHlsChallenge'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 32
mini_batch_size       = 64*4

In [4]:
os.makedirs(rtl_sim_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [5]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [6]:
# define network
if False:
    net = bb.Sequential([
                bb.Binarize(binary_th=0.5, binary_low=0.0, binary_high=1.0),
                bb.DifferentiableLut([256]),
                bb.DifferentiableLut([64*6]),
                bb.AverageLut([64], connection='serial'),
                bb.DifferentiableLut([60*6]),
                bb.AverageLut([60], connection='serial'),
                bb.Reduce([10]),
            ])
#

net = bb.Sequential([
            bb.Binarize(binary_th=0.5, binary_low=0.0, binary_high=1.0),
            bb.DifferentiableLut([1024]),
            bb.DifferentiableLut([512]),
            bb.DifferentiableLut([256]),
            bb.DifferentiableLut([256]),
            bb.DifferentiableLut([128]),
            bb.DifferentiableLut([60*6]),
            bb.AverageLut([60]),
#           bb.AverageLut([60], connection='serial'),
            bb.Reduce([10]),
        ])

net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam(learning_rate=0.0001)

net.print_info()

----------------------------------------------------------------------
[Sequential] 
 input  shape : [1, 28, 28] output shape : [10]
  --------------------------------------------------------------------
  [Binarize] 
   input  shape : {1, 28, 28} output shape : {1, 28, 28}
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {1, 28, 28} output shape : {1024}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {1024} output shape : {512}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {512} output shape : {256}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {256} output shape : {256}
   binary : 1   batch_norm : 1
  -------------------------------

In [7]:
#bb.load_networks(data_path, net)

In [8]:
epochs=64*4

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [9]:
#bb.load_networks(data_path, net)

# learning
optimizer.set_variables(net.get_parameters(), net.get_gradients())
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
#   with tqdm(loader_train) as t:
#       for images, labels in t:
    for images, labels in loader_train:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)

            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)

            net.backward(dy_buf)

            optimizer.update()
        
#           t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)
    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))
    
    bb.save_networks(data_path, net)

epoch[0] : loss=1.258996 accuracy=0.737800
epoch[1] : loss=1.130509 accuracy=0.834800
epoch[2] : loss=1.070093 accuracy=0.872600
epoch[3] : loss=1.056846 accuracy=0.884800
epoch[4] : loss=1.039391 accuracy=0.884900
epoch[5] : loss=1.021478 accuracy=0.888300
epoch[6] : loss=1.048017 accuracy=0.881900
epoch[7] : loss=1.039668 accuracy=0.882500
epoch[8] : loss=1.024815 accuracy=0.896300
epoch[9] : loss=1.016340 accuracy=0.899300
epoch[10] : loss=1.010772 accuracy=0.900000
epoch[11] : loss=1.008850 accuracy=0.899300
epoch[12] : loss=1.005124 accuracy=0.894800
epoch[13] : loss=1.016559 accuracy=0.895300
epoch[14] : loss=1.007699 accuracy=0.903200
epoch[15] : loss=1.020140 accuracy=0.894800
epoch[16] : loss=1.003123 accuracy=0.897600
epoch[17] : loss=1.013102 accuracy=0.899000
epoch[18] : loss=1.031602 accuracy=0.896000
epoch[19] : loss=1.003892 accuracy=0.908200
epoch[20] : loss=0.992965 accuracy=0.908200
epoch[21] : loss=1.004521 accuracy=0.902200
epoch[22] : loss=0.996291 accuracy=0.90000

In [10]:
----------------

SyntaxError: invalid syntax (2069451464.py, line 1)

## FPGA用HLS(C言語高位合成)で使う為の出力

内部データを取得する例としてHSL(C言語高位合成)用の出力を作ってみます

In [ ]:
def make_lut_func_name(name, node):
    return "%s_%d"%(name, node)


def dump_hls_lut_node5(f, name, lut, node):
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    tbl = 0
    for i in range(s):
        if lut.get_lut_table(node ,i):
            tbl += (1 << i)
    f.write("Q(%s,0x%016xLL)\n"%(make_lut_func_name(name, node), tbl))
#   f.write("LF(%s,0x%016x)\n"%(make_lut_func_name(name, node), tbl))

def dump_hls_lut_node4(f, name, lut, node):
#    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    f.write("\nap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    
    tbl = 0
    for i in range(s):
        if lut.get_lut_table(node ,i):
            tbl += (1 << i)
    
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
#   f.write("#pragma HLS inline\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    return ((0x%016xLL >> index) & 1);\n"%tbl)
    f.write("}\n\n")

def dump_hls_lut_node3(f, name, lut, node):
    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
#    f.write("\nap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    
    tbl = 0
    for i in range(s):
        if lut.get_lut_table(node ,i):
            tbl += (1 << i)
    
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
#   f.write("#pragma HLS inline\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    static Lut6Model table(0x%016xLL);\n"%(tbl))
    f.write("    return table.Get(index);\n")
    f.write("}\n\n")

def dump_hls_lut_node2(f, name, lut, node):
    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
    f.write("#pragma HLS inline\n\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    \n")
    f.write("    const ap_uint<1> table[%d] = {"%(s))
    for i in range(s):
        f.write("%d,"%(lut.get_lut_table(node ,i)))
    f.write("};\n")
#    for i in range(s):
#        f.write("    table[%d] = %d;\n"%(i, lut.get_lut_table(node ,i)))
#    f.write("    \n")
#   f.write("    #pragma HLS resource variable=table core=ROM_1P_LUTRAM\n")
    f.write("    #pragma HLS bind_storage variable=table type=ROM_1P impl=LUTRAM\n")
    f.write("    return table[index];\n")
    f.write("}\n\n")

def dump_hls_lut_node1(f, name, lut, node):
    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    
    tbl = 0
    for i in range(s):
        if lut.get_lut_table(node ,i):
            tbl += (1 << i)
    
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
    f.write("#pragma HLS inline\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    const ap_uint<%d> table= 0x%016xLL;\n"%(s, tbl))
    f.write("    return table[index];\n")
    f.write("}\n\n")

def dump_hls_lut(f, name, lut):
    ins  = lut.get_input_node_size()
    outs = lut.get_output_node_size()
    for node in range(outs):
        dump_hls_lut_node5(f, name, lut, node)
    
    f.write("\n")
    f.write("inline ap_uint<%d> %s(ap_uint<%d> i)\n"%(outs, name, ins))
    f.write("{\n")
    f.write("ap_uint<%d>  o;\n"%(outs))
    for node in range(outs):
        f.write("o[%d]=%s("%(node, make_lut_func_name(name, node)))
        n = lut.get_node_connection_size(node)
        for i in range(n):
            f.write("i[%d]"%(lut.get_node_connection_index(node, i)))
            if i < n-1: 
                f.write(",")
            else:
                f.write(");\n")
    f.write("return o;\n")   
    f.write("}\n\n")

# 学習済みを読みなおす
#bb.load_networks(data_path, net)
with open("MnistDifferentiableLutSimpleHls_.h", "w") as f:
#   f.write('#include "ap_int.h"\n\n')
    for i in range(1, 4):
        dump_hls_lut(f, "l%d"%i, net[i])

In [ ]:
W = (net[4].W().quantize(16, 1/256).numpy() * 256).astype(np.int32)
b = (net[4].b().quantize(16, 1/256).numpy() * 256).astype(np.int32)

In [ ]:
print(np.max(W))
print(np.min(W))
print(np.max(b))
print(np.min(b))

In [ ]:
with open("MnistDifferentiableLutSimpleHls2_.h", "w") as f:
    f.write('\n\n')
    f.write('#define WLEN %d\n\n'%W.shape[1])    
    f.write('const ap_int<WBITS> W_tbl[10][WLEN] =\n')
    f.write('\t{\n')
    for i in range(10):
        f.write('\t\t{')
        for j in range(W.shape[1]):
            f.write('%5d, '%W[i][j])
        f.write('\t\t},\n')
    f.write('\t};\n\n')
    
    f.write('const ap_int<BBITS> b_tbl[10] =\n')
    f.write('\t{')
    for i in range(10):
        f.write('%5d, '%b[i])
    f.write('};\n\n')
    


In [ ]:
num = 0
with open('mnist_hls_test_.txt', 'w') as f:
    for images, labels in loader_test:
        x_buf = np.array(images).astype(np.float32)
        t_buf = np.array(labels)
        for i in range(x_buf.shape[0]):
            f.write("%d"%t_buf[i])
            for y in range(x_buf.shape[2]):
                for x in range(x_buf.shape[3]):
                    f.write(" %d"%(x_buf[i, 0, y, x] > 0.5))
            f.write("\n")
            num += 1
        if num > 1024:
            break

In [ ]:
!cp MnistDifferentiableLutSimpleHls_.h ../../../_work/mnist6-free/
!cp MnistDifferentiableLutSimpleHls2_.h ../../../_work/mnist6-free/